In [1]:
import snappy
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 1000)

In [3]:
data = pd.read_parquet("data/train.snappy")

In [4]:
data = pd.DataFrame(data)

In [5]:
data['descripcion'].iloc[4601]

'**La observación** 4604  describe una seta de **clase** poisonous.  Tiene una **forma de sombrero** convex  con una **superficie** scaly  **y un color** brown.  **No presenta**  moretones y tiene un **olor** foul.  **Las branquias son de tipo** free  con una **separación** close  **y un tamaño** narrow.  **El color de las branquias es** buff.  **La forma del tallo** es tapering  y la **raíz del tallo** es ?.  **La superficie del tallo por encima del anillo** es smooth  **y por debajo del anillo** es smooth.  El **color del tallo por encima del anillo** es pink  **y por debajo del anillo** es pink.  **El tipo de velo** es partial  con un **color de velo** white.  Tiene one **anillo(s)**,  **de tipo** evanescent  **y el color de la impresión de esporas** es white.  Se encuentra en una **población** several  y su **hábitat es** paths.  '

In [6]:
data_test = data['descripcion'][0]

In [7]:
data_test

'**La observación** 367  describe una seta de **clase** edible.  Tiene una **forma de sombrero** bell  con una **superficie** scaly  **y un color** yellow.  **Presenta**  moretones y tiene un **olor** almond.  **Las branquias son de tipo** free  con una **separación** close  **y un tamaño** broad.  **El color de las branquias es** black.  **La forma del tallo** es enlarging  y la **raíz del tallo** es club.  **La superficie del tallo por encima del anillo** es smooth  **y por debajo del anillo** es smooth.  El **color del tallo por encima del anillo** es white  **y por debajo del anillo** es white.  **El tipo de velo** es partial  con un **color de velo** white.  Tiene one **anillo(s)**,  **de tipo** pendant  **y el color de la impresión de esporas** es brown.  Se encuentra en una **población** scattered  y su **hábitat es** grasses.  '

In [14]:
import pandas as pd
import re

# Example DataFrame
# df = pd.DataFrame({
#     'raw_text': [
#         "**La observación** 367  describe una seta de **clase** edible.  Tiene una **forma de sombrero** bell  con una **superficie** scaly  **y un color** yellow.  **Presenta** moretones y tiene un **olor** almond.  **Las branquias son de tipo** free  con una **separación** close  **y un tamaño** broad.  **El color de las branquias es** black.  **La forma del tallo** es enlarging y la **raíz del tallo** es club.  **La superficie del tallo por encima del anillo** es smooth **y por debajo del anillo** es smooth.  El **color del tallo por encima del anillo** es white **y por debajo del anillo** es white.  **El tipo de velo** es partial con un **color de velo** white.  Tiene one **anillo(s)**,  **de tipo** pendant  **y el color de la impresión de esporas** es brown.  Se encuentra en una **población** scattered  y su **hábitat es** grasses."
#     ]
# })
df = pd.DataFrame(data)

# Function to extract structured data
def extract_columns(text):
    extracted_data = {}

    # Fix cases where the value comes before the column name (e.g., "Tiene one **anillo(s)**")
    text = re.sub(r'Tiene one\s+\*\*(anillo\(s\))\*\*', r'**\1** one', text)

    # Regular expression to match "**Column Name** Value" pattern
    pattern = re.findall(r'\*\*(.*?)\*\*\s([^*]+)', text)

    for col, val in pattern:
        col = col.strip()  # Clean column name
        val = val.strip()  # Clean value

        # Remove unnecessary words at the start of values
        val = re.sub(r'^(es|de|con una|con un|y un|y una|y su|,|El|Se encuentra en una|su)\s+', '', val)

        # If the value contains another "**column name**", split it to avoid merging data
        val_cleaned = re.split(r'\s\*\*', val, maxsplit=1)[0]

        extracted_data[col] = val_cleaned.strip()
    
    return extracted_data

# Apply extraction function to each row
df_extracted = df['descripcion'].apply(lambda x: pd.Series(extract_columns(x)))


In [15]:
df_extracted = df_extracted.drop(columns=['No presenta', 'anillo(s)**,  **de tipo'])

In [18]:
df_extracted.rename(columns={'La observación':'n_obs', 'forma de sombrero':'forma_sombrero','y un color':'color_sombrero', 'Presenta':'moretones','Las branquias son de tipo':'tipo_branquias','separación':'separacion','y un tamaño':'tamano_branquias','El color de las branquias es':'color_branquias','La forma del tallo':'forma_tallo','raíz del tallo':'raiz_tallo','La superficie del tallo por encima del anillo':'sup_tallo_encima_anillo','y por debajo del anillo':'sup_tallo_debajo_anillo','color del tallo por encima del anillo':'color_tallo_encima_anillo','El tipo de velo':'tipo_velo','color de velo':'color_velo','anillo(s)':'n_anillos','de tipo':'tipo_anillos','y el color de la impresión de esporas':'color_impresion_esporas','población':'poblacion','hábitat es':'habitat'})

,n_obs,clase,forma_sombrero,superficie,color_sombrero,moretones,olor,tipo_branquias,separacion,tamano_branquias,color_branquias,forma_tallo,raiz_tallo,sup_tallo_encima_anillo,sup_tallo_debajo_anillo,color_tallo_encima_anillo,tipo_velo,color_velo,n_anillos,tipo_anillos,color_impresion_esporas,poblacion,habitat
0,367 describe una seta de,edible. Tiene una,bell con una,scaly,yellow.,moretones y tiene un,almond.,free con una,close,broad.,black.,enlarging y la,club.,smooth,white.,white,partial con un,white.,"one,",pendant,brown. Se encuentra en una,scattered y su,grasses.
1,5123 describe una seta de,poisonous. Tiene una,convex con una,smooth,red.,NaN,foul.,free con una,close,narrow.,buff.,tapering y la,?.,silky,pink.,white,partial con un,white.,"one,",evanescent,white. Se encuentra en una,several y su,paths.
2,2864 describe una seta de,edible. Tiene una,convex con una,fibrous,green.,moretones y tiene un,null.,free con una,close,broad.,pink.,tapering y la,bulbous.,smooth,white.,pink,partial con un,white.,"one,",pendant,brown. Se encuentra en una,several y su,wood.
3,4674 describe una seta de,poisonous. Tiene una,flat con una,smooth,white.,moretones y tiene un,foul.,free con una,close,broad.,pink.,tapering y la,bulbous.,fibrous,white.,white,partial con un,white.,"one,",pendant,chocolate. Se encuentra en una,scattered y su,grasses.
4,1210 describe una seta de,edible. Tiene una,convex con una,smooth,white.,NaN,null.,free con una,crowded,broad.,black.,tapering y la,equal.,fibrous,white.,white,partial con un,white.,"one,",evanescent,black. Se encuentra en una,abundant y su,grasses.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4598,1028 describe una seta de,edible. Tiene una,convex con una,fibrous,brown.,moretones y tiene un,null.,free con una,close,broad.,purple.,tapering y la,bulbous.,smooth,gray.,gray,partial con un,white.,"one,",pendant,black. Se encuentra en una,several y su,wood.
4599,5591 describe una seta de,poisonous. Tiene una,convex con una,smooth,red.,NaN,fishy.,free con una,close,narrow.,buff.,tapering y la,?.,smooth,white.,pink,partial con un,white.,"one,",evanescent,white. Se encuentra en una,several y su,wood.
4600,2053 describe una seta de,edible. Tiene una,convex con una,fibrous,red.,moretones y tiene un,null.,free con una,close,broad.,brown.,tapering y la,bulbous.,smooth,gray.,white,partial con un,white.,"one,",pendant,brown. Se encuentra en una,solitary y su,wood.
4601,4604 describe una seta de,poisonous. Tiene una,convex con una,scaly,brown.,NaN,foul.,free con una,close,narrow.,buff.,tapering y la,?.,smooth,pink.,pink,partial con un,white.,"one,",evanescent,white. Se encuentra en una,several y su,paths.


In [20]:
df_extracted.head()

,La observación,clase,forma de sombrero,superficie,y un color,Presenta,olor,Las branquias son de tipo,separación,y un tamaño,El color de las branquias es,La forma del tallo,raíz del tallo,La superficie del tallo por encima del anillo,y por debajo del anillo,color del tallo por encima del anillo,El tipo de velo,color de velo,anillo(s),de tipo,y el color de la impresión de esporas,población,hábitat es
0,367 describe una seta de,edible. Tiene una,bell con una,scaly,yellow.,moretones y tiene un,almond.,free con una,close,broad.,black.,enlarging y la,club.,smooth,white.,white,partial con un,white.,"one,",pendant,brown. Se encuentra en una,scattered y su,grasses.
1,5123 describe una seta de,poisonous. Tiene una,convex con una,smooth,red.,NaN,foul.,free con una,close,narrow.,buff.,tapering y la,?.,silky,pink.,white,partial con un,white.,"one,",evanescent,white. Se encuentra en una,several y su,paths.
2,2864 describe una seta de,edible. Tiene una,convex con una,fibrous,green.,moretones y tiene un,null.,free con una,close,broad.,pink.,tapering y la,bulbous.,smooth,white.,pink,partial con un,white.,"one,",pendant,brown. Se encuentra en una,several y su,wood.
3,4674 describe una seta de,poisonous. Tiene una,flat con una,smooth,white.,moretones y tiene un,foul.,free con una,close,broad.,pink.,tapering y la,bulbous.,fibrous,white.,white,partial con un,white.,"one,",pendant,chocolate. Se encuentra en una,scattered y su,grasses.
4,1210 describe una seta de,edible. Tiene una,convex con una,smooth,white.,NaN,null.,free con una,crowded,broad.,black.,tapering y la,equal.,fibrous,white.,white,partial con un,white.,"one,",evanescent,black. Se encuentra en una,abundant y su,grasses.


In [19]:
pd.read_parquet(r'C:\Users\Felipe\Downloads\train (1).parquet')

,id,class,cap_shape,cap_surface,cap_color,odor,gill_attachment,gill_spacing,gill_size,gill_color,stalk_shape,stalk_root,stalk_surface_above_ring,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat,bruises
0,367,edible,bell,scaly,yellow,almond,free,close,broad,black,enlarging,club,smooth,smooth,white,white,partial,white,one,pendant,brown,scattered,grasses,Bruises
1,5123,poisonous,convex,smooth,red,foul,free,close,narrow,buff,tapering,?,silky,silky,white,pink,partial,white,one,evanescent,white,several,paths,No presenta
2,2864,edible,convex,fibrous,green,null,free,close,broad,pink,tapering,bulbous,smooth,smooth,pink,white,partial,white,one,pendant,brown,several,wood,Bruises
3,4674,poisonous,flat,smooth,white,foul,free,close,broad,pink,tapering,bulbous,fibrous,fibrous,white,white,partial,white,one,pendant,chocolate,scattered,grasses,Bruises
4,1210,edible,convex,smooth,white,null,free,crowded,broad,black,tapering,equal,fibrous,fibrous,white,white,partial,white,one,evanescent,black,abundant,grasses,No presenta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4598,1028,edible,convex,fibrous,brown,null,free,close,broad,purple,tapering,bulbous,smooth,smooth,gray,gray,partial,white,one,pendant,black,several,wood,Bruises
4599,5591,poisonous,convex,smooth,red,fishy,free,close,narrow,buff,tapering,?,smooth,silky,pink,white,partial,white,one,evanescent,white,several,wood,No presenta
4600,2053,edible,convex,fibrous,red,null,free,close,broad,brown,tapering,bulbous,smooth,smooth,white,gray,partial,white,one,pendant,brown,solitary,wood,Bruises
4601,4604,poisonous,convex,scaly,brown,foul,free,close,narrow,buff,tapering,?,smooth,smooth,pink,pink,partial,white,one,evanescent,white,several,paths,No presenta
